<a href="https://colab.research.google.com/github/ryanoleary26/VirusTotal-Payload-Scanner/blob/master/Honeypot_Analyse_Payloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing Honeypot Payloads with the VirusTotal API

Any files that are given to this script in a .ZIP file will be indivually sent to VirusTotal for scanning using [their v2 API](https://developers.virustotal.com/reference).

#Features



1. Displays scan reports from over [70 different Antivirus products](https://support.virustotal.com/hc/en-us/articles/115002146809-Contributors).
2. Provides a URL to each indivudual scan performed.
3. Provides statitistics on total scans performed and total positive results.
4. Ideal for analysis of malicious files collected through honeypots.
5. Basic API quota usage detection
6. Option to show scan results from specific Antivirus scanners instead of an overall summary.

#Prerequesites

1. Python
2. [Requests](https://requests.readthedocs.io/en/master/) module

#How to use

1. To use this script for yourself you will need to obtain a free public API key by [creating an account on the VirusTotal website](https://www.virustotal.com/gui/join-us).
2. Enter your API key on line 8, see below:

  ```
  #Enter API key here ########
  api_key = '<API KEY HERE>' #
  ############################
  ```
3. Enter the file path of the ZIP file you wish to use on line 16, see below. Note that currentlyy files that are contained within sub-directories of the ZIP file are currently not scanned. To get round this seperate groups of files into seperate ZIP files until I implement this feature.
```
  #Enter the location of your .ZIP file here ######
  with ZipFile('<FILE PATH HERE>', 'r') as zipObj:#
  #################################################
```
4. Line 12 contains a dictionary of Antivirus (AV) names to include in the scan output. Add or remove Antivirus names from this dictionary as you please. The default configuration will include Avast, AVG, BitDefender, FireEye, F-Secure, McAfee and Microsoft in the specific scan result output. This **IS** case sensitive.
```
#Customise the scan output by changing the included AV names here. This IS case and space sensitive.
antiviruses = {'Avast', 'AVG', 'BitDefender', 'FireEye', 'F-Secure', 'Malwarebytes', 'McAfee', 'Microsoft'}
```
5. Run the script and enjoy the sexy Antivirus results - Happy Virus Hunting!








In [12]:
import json
from zipfile import ZipFile
import time
import requests
import os
import sys

#Enter API key here ##########################################################
api_key = '8d9600ee37bd48859cc0fbbe597f56c73ed197f52f869610b0b8f445637df79c' #
##############################################################################  

#Customise the scan output by changing the included AV names here. This IS case and space sensitive.########
antiviruses = {'Avast', 'AVG', 'BitDefender', 'FireEye', 'F-Secure', 'Malwarebytes', 'McAfee', 'Microsoft'}#
############################################################################################################
try:
  #Enter the location of your .ZIP file here ###########################
  with ZipFile('/content/drive/My Drive/Downloads.zip', 'r') as zipObj:#
  ######################################################################

    # Extract all the contents of zip file in different directory
    hashList = zipObj.namelist()

  for payload in hashList:
      #Makes request to virustotal to scan file
      url = 'https://www.virustotal.com/vtapi/v2/file/report'
      params = {'apikey': api_key, 'resource': payload}
      try:
        response = requests.get(url, params=params)
        #print('HTTP Response Code:', response.status_code)
        vt_response = response.json()
      except:
        if response.status_code == 204:
        #Handles the 204 response code
          print('Recieved HTTP Response Code:', response.status_code, '-- Indicates that API usage quota has been used up.\nRefer to the VirusTotal API documentation for error description.\nDocumentation link: https://developers.virustotal.com/reference#api-responses')
          break
        else:
          print('JSON Decoder Error')
          break
      
      #Parsing the JSON response
      print('Scanning Payload: ', payload)
      if vt_response['response_code'] == 1:
        print('Total Scans:', vt_response['total'])
        print('=== Scan Results ===')
        for av in antiviruses: #Loops through selected AVs for detection results
          try:
            print(av, '-> Detected:', vt_response['scans'][av]['detected'], ' Result:', vt_response['scans'][av]['result'])
          except KeyError:
            #AVs included in 'antiviruses' might not always return scan results, which will cause a TypeError
            print(av, 'could not return any test results.')
        print('====================')
        print('Total Positive Results:', vt_response['positives'])
        print('Scan Permalink: ', vt_response['permalink'], '\n')
      
      #Handles other response codes
      elif vt_response['response_code'] != 1:
        print('=======================================================\n',vt_response['verbose_msg'],'\n')

      #Sleeps script to comply with API user agreement
      time.sleep(15)

#Generic exception handling :/
except Exception as error:
  print(error)
  
print('Scan complete')

Scanning Payload:  0013d4e2ee63e2be651c99071bb3207c5acef4b9aa1e135e8f19eba656e68d1a
Total Scans: 58
=== Scan Results ===
F-Secure -> Detected: False  Result: None
AVG -> Detected: False  Result: None
Microsoft -> Detected: False  Result: None
FireEye -> Detected: False  Result: None
BitDefender -> Detected: False  Result: None
Malwarebytes -> Detected: False  Result: None
McAfee -> Detected: False  Result: None
Avast -> Detected: False  Result: None
Total Positive Results: 0
Scan Permalink:  https://www.virustotal.com/file/0013d4e2ee63e2be651c99071bb3207c5acef4b9aa1e135e8f19eba656e68d1a/analysis/1588351732/ 



KeyboardInterrupt: ignored